In [35]:
from pynq.overlays.base import BaseOverlay
import time
base = BaseOverlay("base.bit")
btns = base.btns_gpio

Task was destroyed but it is pending!
task: <Task pending name='Task-3' coro=<flash_leds() running at /tmp/ipykernel_2406/3564656741.py:9> wait_for=<Future pending cb=[Task.__wakeup()]>>
Task was destroyed but it is pending!
task: <Task pending name='Task-4' coro=<get_btns() running at /tmp/ipykernel_2406/3564656741.py:29> wait_for=<Future pending cb=[Task.__wakeup()]>>
Task was destroyed but it is pending!
task: <Task pending name='Task-13' coro=<flash_leds() running at /tmp/ipykernel_2406/1119625195.py:11> wait_for=<Future pending cb=[Task.__wakeup()]>>
Task was destroyed but it is pending!
task: <Task pending name='Task-14' coro=<get_btns() running at /tmp/ipykernel_2406/1119625195.py:34> wait_for=<Future pending cb=[Task.__wakeup()]>>
Task was destroyed but it is pending!
task: <Task pending name='Task-23' coro=<flash_leds() running at /tmp/ipykernel_2406/3182707231.py:14> wait_for=<Future pending cb=[Task.__wakeup()]>>


In [46]:
%%microblaze base.PMODA

#include "gpio.h"
#include "pyprintf.h"

const unsigned int R=3,G=2,B=1,HIGH=1,LOW=0;

// Channel 1 PMODA D1-D3
gpio parent = gpio_open_device(0);
gpio ledPins = gpio_configure(parent, B, R, 1);  // B = D1, G=D2, R=D3


//Function to turn on/off a selected pin of PMODA
void write_gpio(unsigned int pin, unsigned int val){
    if(pin>7){
        pyprintf("D0-D7 only");
        return;
    }
    if (val > 1){
        pyprintf("pin value must be 0 or 1");
        return;
    }
    gpio pin_out = gpio_open(pin);
    gpio_set_direction(pin_out, GPIO_OUT);
    gpio_write(pin_out, val);
    pyprintf("set write pin: %d \r\n",pin);
}

//Function to read the value of a selected pin of PMODA
unsigned int read_gpio(unsigned int pin){
    if(pin>7){
        pyprintf("D0-D7 only");
        return 0;
    }
    
    gpio pin_in = gpio_open(pin);
    gpio_set_direction(pin_in, GPIO_IN);
    
    pyprintf("set read pin: %d\r\n", pin);
    return gpio_read(pin_in);
}

int read_only(){
    return gpio_read(ledPins) & 0b111;
}

int read_state_gpio(unsigned int pin){
    if( pin==0 || pin>3){
        pyprintf("D1-D3 only");
        return -1;
    }
    gpio pin_in = gpio_open(pin);
    return gpio_read(pin_in);
}

int init(){
    pyprintf("init \r\n");
    for(int i=0; i<8; ++i){
        read_gpio(i); // read only
    }
    return 0b1111+0b1111;// top two rows
}

void send_tx_handle(unsigned int val){
    if(val > 7){ 
        pyprintf("val must be 0..7\r\n"); // since three bits (0b111) can only be max 7 in decimal
        return;
    }
    gpio_set_direction(ledPins, GPIO_OUT);
    gpio_write(ledPins, val); // LSB is D1, so val is D3D2D1
    //pyprintf("TX wrote %d\r\n", val);
}

In [4]:
init()

init 
set read pin: 0
set read pin: 1
set read pin: 2
set read pin: 3
set read pin: 4
set read pin: 5
set read pin: 6
set read pin: 7


30

In [5]:
# OFF 0x000 since no bits are powered
send_tx_handle(0)

In [6]:
# Blue since B is the first bit in RGB => 0b001 <=> 1 decimal 
send_tx_handle(1)
read_only()

TX wrote 0
TX wrote 1


1

In [7]:
# Green since G is the second bit in RGB => 0b010 <=> 2 decimal
send_tx_handle(2)
read_only()

TX wrote 2


2

In [8]:
# Red since R is the third bit in RGB => 0b100 <=> 4 decimal
send_tx_handle(4)
read_only()

TX wrote 4


4

In [49]:
# Using asyncio for PMODA pins
import asyncio
cond = True
start = True
color = 0x7 # white

async def flash_leds():
    global cond, start
    while cond:
        send_tx_handle(0x0)
        await asyncio.sleep(0.25)

        send_tx_handle(color)
        await asyncio.sleep(0.25)

async def get_btns(_loop):
    global cond, start, color
    while cond:
        await asyncio.sleep(0.01)
        if btns[0].read() != 0:
            color = 0x4
        if btns[1].read() != 0:
            color = 0x2
        if btns[2].read() != 0:
            color = 0x1
        if btns[3].read() != 0:
            _loop.stop()
            cond = False

loop = asyncio.new_event_loop()
loop.create_task(flash_leds())
loop.create_task(get_btns(loop))
loop.run_forever()
loop.close()
send_tx_handle(0x0)
send_tx_handle(0x0)
print("Done.")

Done.
